In [158]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plth
from numpy.random import randn
from scipy.stats import norm
from numpy.linalg import inv
#data
#slide 21 1 tabel
countries = ['Australia','Canada','France','Germany','Japan','UK','USA']
cor = np.matrix('1 0.488 0.478 0.515 0.439 0.512 0.491;0.488 1 0.664 0.655 0.310 0.608 0.779;0.478 0.664 1 0.861 0.355 0.783 0.668;0.515 0.655 0.861 1 0.354 0.777 0.653;0.439 0.310 0.355 0.354 1 0.405 0.306;0.512 0.608 0.783 0.777 0.405 1 0.652;0.491 0.779 0.668 0.653 0.306 0.652 1')
sig = np.matrix([0.16,0.203,0.248,0.271,0.210,0.200,0.187])
weq =np.matrix([0.016,0.022,0.052,0.055,0.116,0.124,0.615])
eqrp = np.matrix([0.039,0.069,0.084,0.09,0.043,0.068,0.076])
covar =np.multiply(np.multiply(sig.T,cor),sig)
gamma = 3

#a, formula slide 16)
mu0= gamma* weq @ covar
mu0
pd.DataFrame(mu0.T,index = countries, columns =['mu0']).round(5)


,mu0
Australia,0.04725
Canada,0.08298
France,0.10030
Germany,0.10833
Japan,0.05163
UK,0.08121
USA,0.09072


In [159]:
#b formula slide 15)
gamma1 =2.5
tau= 0.05
mu1=gamma1* weq @ covar
w0 = mu1 @np.linalg.inv(gamma1*(1+tau)*covar)
pd.DataFrame(np.hstack([w0.T,(w0-weq).T]),index = countries, columns =['w0','w0-weq']).round(5)


,w0,w0-weq
Australia,0.01524,-0.00076
Canada,0.02095,-0.00105
France,0.04952,-0.00248
Germany,0.05238,-0.00262
Japan,0.11048,-0.00552
UK,0.11810,-0.00590
USA,0.58571,-0.02929


In [170]:
#c same investor as in part b)
#BL slide 13
def BL(omega0,P,omega,mu,q):
    mu_p = mu.T + omega0@P.T@inv(P@omega0@P.T+omega)@(q.T-P@mu.T)
    omega_p =omega0-omega0@P.T@inv(P@omega0@P.T+omega)@P@omega0
    return mu_p,omega_p

omega0 = tau*covar
P = np.matrix([[0, 0,-weq[0,2]/(weq[0,2]+weq[0,5]),1,0,-weq[0,5]/(weq[0,2]+weq[0,5]),0 ],[0, 1, 0, 0, 0, 0, -1]]) #2x7 matrix
omega= np.matrix([[0.021*tau ,0], [0, 0.017*tau]])
mu = mu1
q = np.matrix([0.06 ,0.02])
mu_p,omega_p = BL(omega0,P,omega,mu,q)
mu_p
pd.DataFrame(mu_p,index= countries, columns= ['mu_bar'])

,mu_bar
Australia,0.045193
Canada,0.085961
France,0.097199
Germany,0.118030
Japan,0.046486
UK,0.070255
USA,0.078209


In [164]:
pd.DataFrame(omega_p,index=countries,columns =countries)

,Australia,Canada,France,Germany,Japan,UK,USA
Australia,0.001247,0.000724,0.000870,0.000951,0.000720,0.000804,0.000701
Canada,0.000724,0.001784,0.001513,0.001498,0.000615,0.001206,0.001503
France,0.000870,0.001513,0.002890,0.002500,0.000882,0.001906,0.001466
Germany,0.000951,0.001498,0.002500,0.002827,0.000920,0.002028,0.001456
Japan,0.000720,0.000615,0.000882,0.000920,0.002195,0.000842,0.000588
UK,0.000804,0.001206,0.001906,0.002028,0.000842,0.001993,0.001201
USA,0.000701,0.001503,0.001466,0.001456,0.000588,0.001201,0.001646


In [165]:
#check with equations 
mu_check = inv(inv(omega0)+P.T@inv(omega)@P)@(inv(omega0)@mu.T+P.T@inv(omega)@q.T)
omega_check=inv(inv(omega0)+P.T@inv(omega)@P)
print(mu_check)
print(omega_check)

[[0.04519308]
 [0.08596128]
 [0.0971992 ]
 [0.11803025]
 [0.04648631]
 [0.07025482]
 [0.07820875]]
[[0.00124701 0.0007237  0.00087028 0.00095084 0.00071954 0.0008039
  0.00070082]
 [0.0007237  0.00178412 0.00151282 0.00149759 0.0006151  0.00120575
  0.00150308]
 [0.00087028 0.00151282 0.00289031 0.00249987 0.00088213 0.00190552
  0.00146613]
 [0.00095084 0.00149759 0.00249987 0.0028269  0.00091955 0.00202775
  0.00145562]
 [0.00071954 0.0006151  0.00088213 0.00091955 0.0021947  0.00084237
  0.00058828]
 [0.0008039  0.00120575 0.00190552 0.00202775 0.00084237 0.00199281
  0.00120115]
 [0.00070082 0.00150308 0.00146613 0.00145562 0.00058828 0.00120115
  0.00164628]]


In [166]:
#d formula slide 15)
w_star = inv(gamma1*(covar+omega_p))@mu_p
pd.DataFrame(np.hstack([w_star,w_star-w0.T,w_star-weq.T]),index= countries, columns = ['w_star','w_star-w0','w_star-weq'])


,w_star,w_star-w0,w_star-weq
Australia,0.015238,1.908196e-16,-0.000762
Canada,0.298641,2.776884e-01,0.276641
France,-0.063789,-1.133128e-01,-0.115789
Germany,0.435901,3.835202e-01,0.380901
Japan,0.110476,0.000000e+00,-0.005524
UK,-0.152112,-2.702074e-01,-0.276112
USA,0.308026,-2.776884e-01,-0.306974


In [167]:
#e)
#lambda slide 18
Lam = 1/gamma1*inv(P@covar@P.T+(1+1/tau)*omega)@(q.T-1/(1+tau)*P@mu1.T)
Lam
pd.DataFrame(Lam.T,columns=['Lambda1','Lambda2'])

,Lambda1,Lambda2
0,0.38352,0.277688


In [168]:
#f)
P1 = np.matrix([[0, 0,-weq[0,2]/(weq[0,2]+weq[0,5]),1,0,-weq[0,5]/(weq[0,2]+weq[0,5]),0 ],[0, 1, 0, 0, 0, 0, -1],[0,0,0,0,1,0,0]]) #2x7 matrix
omega1= np.matrix([[0.021*tau ,0,0], [0, 0.017*tau,0],[0,0,tau*0.03]])
q1 =np.matrix([0.06 ,0.02,0.045])
Lam1 = 1/gamma1*inv(P1@covar@P1.T+(1+1/tau)*omega1)@(q1.T-1/(1+tau)*P1@mu1.T)
pd.DataFrame(Lam1.T,columns=['Lambda1','Lambda2','Lambda3'])

,Lambda1,Lambda2,Lambda3
0,0.383326,0.27759,0.003186


In [169]:
mu_p1,omega_p1=BL(omega0,P1,omega1,mu,q1)
w_star1 = inv(gamma1*(covar+omega_p1))@mu_p1
pd.DataFrame(w_star1,index= countries, columns = ['w_star'])


,w_star
Australia,0.015238
Canada,0.298542
France,-0.063732
Germany,0.435707
Japan,0.113662
UK,-0.151976
USA,0.308124
